In [ ]:
# AF2 weight download (avoid storing in the Docker image)
!mkdir -p weights && \
cd weights && \
wget https://storage.googleapis.com/alphafold/alphafold_params_2022-12-06.tar && \
tar -xvf alphafold_params_2022-12-06.tar && \
rm alphafold_params_2022-12-06.tar && \
cd ..

## PyRosetta Manual Step

You must manually download PyRosetta before continuing:

1. Visit: https://www.pyrosetta.org/home/licensing-pyrosetta
2. Register for academic use
3. Download `.conda` file (e.g. `pyrosetta-2025.22+release.957f89124e.conda`)
    Go to `https://conda.graylab.jhu.edu`  and for example, navigate to the following:
    `https://conda.graylab.jhu.edu/linux-64/rosetta-2025.24+release.8e1e5e54f0-0.conda`
4. Place it in the **same folder** as this notebook.
5. Do not rename it.

The next cell will mount and install it inside the container.

**Do not share or redistribute the file**. This step must be done manually due to license restrictions.


In [ ]:
%%bash
docker run --gpus all -dit --name bindcraft_session \
  -v $(pwd)/targets:/app/bindcraft/targets \
  -v $(pwd)/output:/app/bindcraft/output \
  -v $(pwd)/weights:/app/bindcraft/params \
  -v $(pwd)/pyrosetta-2025.22+release.957f89124e.conda:/app/bindcraft/pyrosetta.conda \
  bindcraft:latest bash

In [ ]:
%%bash
docker exec -u root bindcraft_session bash -c "
  source /opt/conda/etc/profile.d/conda.sh && \
  conda activate bindcraft && \
  mkdir -p /app/pyrosetta && \
  cd /app/pyrosetta && \
  mamba install pyrosetta -c https://conda.graylab.jhu.edu -y"


In [ ]:
%%bash
docker exec bindcraft_session bash -c "
  source /opt/conda/etc/profile.d/conda.sh && \
  conda activate bindcraft && \
  conda install /app/bindcraft/pyrosetta.conda -y"

In [ ]:
%%bash
docker exec bindcraft_session bash -c "
  source /opt/conda/etc/profile.d/conda.sh && \
  conda activate bindcraft && \
  python /app/bindcraft/bindcraft.py \
    --settings /app/bindcraft/targets/my_target.json \
    --filters /app/bindcraft/settings_filters/default_filters.json \
    --advanced /app/bindcraft/settings_advanced/default_4stage_multimer.json"

In [ ]:

import json
import ipywidgets as widgets
from IPython.display import display

# Example JSON settings dict (like what you'd have in your settings file)
settings = {
    "binder_length": 50,
    "temperature": 300,
    "use_multimer": True,
    "target_name": "PDL1"
}

# Create widgets matching those fields
binder_length_widget = widgets.IntSlider(value=settings["binder_length"], min=10, max=100, description='Binder Length')
temperature_widget = widgets.IntText(value=settings["temperature"], description='Temperature (K)')
use_multimer_widget = widgets.Checkbox(value=settings["use_multimer"], description='Use Multimer')
target_name_widget = widgets.Text(value=settings["target_name"], description='Target Name')

# Display widgets
display(binder_length_widget, temperature_widget, use_multimer_widget, target_name_widget)

# Button to save the current widget values back to JSON file
save_button = widgets.Button(description="Save Settings")

output = widgets.Output()

def save_clicked(b):
    # Update dict from widget values
    settings["binder_length"] = binder_length_widget.value
    settings["temperature"] = temperature_widget.value
    settings["use_multimer"] = use_multimer_widget.value
    settings["target_name"] = target_name_widget.value
    
    # Save to JSON file
    with open("settings_output.json", "w") as f:
        json.dump(settings, f, indent=2)
    
    with output:
        output.clear_output()
        print("Settings saved to settings_output.json")

save_button.on_click(save_clicked)

display(save_button, output)


ModuleNotFoundError: No module named 'ipywidgets'